# Stacked Networks Time Series

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import theano
import theano.tensor as T
import cPickle as pickle
from math import sqrt
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import StandardScaler
from sklearn.cross_validation import train_test_split
from itertools import izip
from theano.sandbox.rng_mrg import MRG_RandomStreams as RandomStreams
import theano
%matplotlib inline

### Error Metric

In [2]:
def get_Strong_Error_Percentage(error_list):
    miss = 0
    for dist in error_list:
        if dist > 30.0:
            miss += 1
    return miss/ float(len(error_list))

### Import Data

In [3]:
def formatter(dataList, histLength=40, predLag=6, trainFrac=0.80):
    # Embedding Dimensions, d = 40
    # define pointer that starts at histLength and runs to length - predLag
    attr = []
    y = []
    
    for iPoint in range(histLength, len(dataList) - predLag):
        # flatten history before iPoint and calculate change in closing price between iPoint and iPoint + predLag
        attrLine = []
        
        for i in range(iPoint-histLength, iPoint):
            # flattens histLength many rows into a single row
            attrLine += dataList[i]
            
        # attrLine = [temp + dataList[i] for i in range(iPoint-histLength, iPoint)]
        attr.append(attrLine)
        currClse = dataList[iPoint][0]          # only get sunspots
        futClse = dataList[iPoint + predLag][0] # only get sunspots
        yVal = futClse - currClse
        y.append(yVal)  #difference in closing prices
        
    #calculate index for start of training set
    #trainStart = int(trainFrac * len(attr))
    #xTr = attr[:trainStart]; xTe = attr[trainStart:]; yTr = y[:trainStart]; yTe = y[trainStart:]
    #xTr = np.array(attr)
    #yTr = np.array(y)
    #attr = MaxAbsScaler().fit_transform(attr)
    return  attr, y    #take raw pricing data, return numpy array of attributes and labels

In [4]:
df = pd.read_csv("./SN_m_tot_V2.0.csv",sep = ";", header = None)

### Format Data

In [5]:
x_past_datas_sunspots, y_true_future_sunspot = formatter(df.values[:,3:4].tolist())

In [8]:
np.array(x_past_datas_sunspots).shape

(3157, 40)

In [9]:
np.array(y_true_future_sunspot).shape

(3157,)

In [6]:
xTrain, xTest, yTrain, yTest = train_test_split(x_past_datas_sunspots,
                                                y_true_future_sunspot,
                                                test_size = 0.15,
                                               random_state = 1)

### Scale Data

In [7]:
# StandardScaler centers data around 0
x_ss = StandardScaler().fit_transform(xTrain)
# MaxAbsScaler scales data between [-1,1]
xtrain_mas = MaxAbsScaler().fit_transform(x_ss)

In [8]:
xtest_ss = StandardScaler().fit_transform(xTest)
xtest_mas = MaxAbsScaler().fit_transform(xtest_ss)

In [9]:
print "zero mean centred and scaled data [-1,1]"
print np.mean(xtest_mas)
print  np.max(xtest_mas)
print np.min(xtest_mas)

zero mean centred and scaled data [-1,1]
7.49517653755e-18
1.0
-0.423525344255


In [10]:
Xtrain = np.array(xtrain_mas, dtype = theano.config.floatX)
ytrain = np.array(yTrain, dtype = theano.config.floatX)
Xtest = np.array(xtest_mas, dtype = theano.config.floatX)
Ytest = np.array(yTest, dtype = theano.config.floatX)

### Stack FC and SLTM Networks

In [15]:
class Stacked_Network(object):

    def __init__(self, nin, n_hidden, nout, wh_dim, wh2_dim, wh3_dim):
        rng = np.random.RandomState(1234)
        # cell input
        W_ug = np.asarray(rng.normal(size=(nin, n_hidden), scale= .01, loc = 0.0), dtype = theano.config.floatX)
        W_hg = np.asarray(rng.normal(size=(n_hidden, n_hidden), scale=.01, loc = 0.0), dtype = theano.config.floatX)
        b_g = np.zeros((n_hidden,), dtype=theano.config.floatX)
        # input gate equation
        W_ui = np.asarray(rng.normal(size=(nin, n_hidden), scale =.01, loc=0.0), dtype = theano.config.floatX)
        W_hi = np.asarray(rng.normal(size=(n_hidden, n_hidden), scale =.01, loc=0.0), dtype = theano.config.floatX)
        b_i = np.zeros((n_hidden,), dtype=theano.config.floatX)
        # forget gate equations
        W_uf = np.asarray(rng.normal(size=(nin, n_hidden), scale =.01, loc=0.0), dtype = theano.config.floatX)
        W_hf = np.asarray(rng.normal(size=(n_hidden, n_hidden), scale =.01, loc=0.0), dtype = theano.config.floatX)
        b_f = np.zeros((n_hidden,), dtype=theano.config.floatX)
        # cell output gate equations
        W_uo = np.asarray(rng.normal(size=(nin, n_hidden), scale =.01, loc=0.0), dtype = theano.config.floatX)
        W_ho = np.asarray(rng.normal(size=(n_hidden, n_hidden), scale =.01, loc=0.0), dtype = theano.config.floatX)
        b_o = np.zeros((n_hidden,), dtype=theano.config.floatX)
        # output layer
        W_hy = np.asarray(rng.normal(size=(n_hidden, nout), scale =.01, loc=0.0), dtype = theano.config.floatX)
        b_hy = np.zeros((nout,), dtype=theano.config.floatX)

        # cell input
        W_ug = theano.shared(W_ug, 'W_ug')
        W_hg = theano.shared(W_hg, 'W_hg')
        b_g = theano.shared(b_g, 'b_g')
        # input gate equation
        W_ui = theano.shared(W_ui, 'W_ui')
        W_hi = theano.shared(W_hi, 'W_hi')
        b_i = theano.shared(b_i, 'b_i')
        # forget gate equations
        W_uf = theano.shared(W_uf, 'W_uf')
        W_hf = theano.shared(W_hf, 'W_hf')
        b_f = theano.shared(b_f, 'b_f')
        # cell output gate equations
        W_uo = theano.shared(W_uo, 'W_uo')
        W_ho = theano.shared(W_ho, 'W_ho')
        b_o = theano.shared(b_o, 'b_o')
        # output layer
        W_hy = theano.shared(W_hy, 'W_hy')
        b_hy = theano.shared(b_hy, 'b_hy')

        
        def floatX(X):
            return np.asarray(X, dtype=theano.config.floatX)
         
        b_h = np.asarray(rng.normal(size=((wh_dim[1],)), scale =.01, loc=0.0), dtype = theano.config.floatX)
        b_h = theano.shared(b_h, 'b_h')
        
    
        b_h2 = np.asarray(rng.normal(size=((wh2_dim[1],)), scale =.01, loc=0.0), dtype = theano.config.floatX)
        b_h2 = theano.shared(b_h2, 'b_h2')
        
        b_h3 = np.asarray(rng.normal(size=((wh3_dim[1],)), scale =.01, loc=0.0), dtype = theano.config.floatX)
        b_h3 = theano.shared(b_h3, 'b_h3')
        
        (h, w) = wh_dim
        w_h = np.asarray(rng.normal(size=((h, w)), scale =.01, loc=0.0), dtype = theano.config.floatX)
        w_h = theano.shared(w_h, 'w_h')
        
        (h, w) = wh2_dim
        w_h2 = np.asarray(rng.normal(size=((h, w)), scale =.01, loc=0.0), dtype = theano.config.floatX)
        w_h2 = theano.shared(w_h2, 'w_h2')
        
        (h, w) = wh3_dim
        w_h3 = np.asarray(rng.normal(size=((h, w)), scale =.01, loc=0.0), dtype = theano.config.floatX)
        w_h3 = theano.shared(w_h3, 'w_h2')

        self.activ1 = T.nnet.sigmoid
        self.activ2 = T.tanh
        
        lr = T.scalar()
        u = T.matrix()
        t = T.scalar()


        
        
        def rectify(X):
            #return T.maximum(X, 0.)
            return T.maximum(X, 0.01*(T.exp(X)-1))  #exponential linear rectifer
            #return T.maximum(X, 0.01*X)  #leaky rectifier
        
        def model(X, w_h,w_h2,w_h3, b_h, b_h2, b_h3):
            h = rectify(T.dot(X, w_h) + b_h)

            h2 = rectify(T.dot(h, w_h2) + b_h2)

            h3 = rectify(T.dot(h2, w_h3) + b_h3)

            return h3
        
        
        
        def adaGrad(cost, params, eta=0.1, epsilon=1e-6):
            grads = T.grad(cost=cost, wrt=params)
            updates = []
            for p, g in zip(params, grads):
                sumGSq = theano.shared(p.get_value() * 0.)
                sumGSq_new = sumGSq + g ** 2
                gradient_scaling = T.sqrt(sumGSq_new + epsilon)
                g = g / gradient_scaling
                updates.append((sumGSq, sumGSq_new))
                updates.append((p, p - eta * g))
            return updates
        
        h0_tm1 = theano.shared(np.zeros(n_hidden, dtype=theano.config.floatX))
        s0_tm1 = theano.shared(np.zeros(n_hidden, dtype=theano.config.floatX))
        
        u1 = model(u, 
                  w_h,
                  w_h2,
                  w_h3,
                  b_h, 
                  b_h2,
                  b_h3)

        #theano.printing.debugprint([h0_tm1, u, W_hh, W_uh, W_hy, b_hh, b_hy], print_type=True)
        [h, s], _ = theano.scan(self.recurrent_fn, sequences = u1,
                           outputs_info = [h0_tm1, s0_tm1],
                           non_sequences = [W_ug, W_hg, b_g, W_ui, W_hi,
                                            b_i, W_uf, W_hf, b_f, W_uo, W_ho, b_o, W_hy, b_hy])

        

        
        # Output Layer
        y = T.dot(h[-1], W_hy) + b_hy
        cost = ((t - y)**2).mean(axis=0).sum()

        gw_h,gw_h2,gw_h3,gb_h, gb_h2,gb_h3,\
        gW_ug, gW_hg, gb_g, gW_ui, gW_hi, gb_i, \
        gW_uf, gW_hf, gb_f, gW_uo, gW_ho, gb_o, gW_hy, gb_hy \
            = T.grad(cost, [ w_h,w_h2,w_h3, b_h, b_h2,b_h3,\
                             W_ug, W_hg, b_g, W_ui, W_hi, b_i, \
                             W_uf, W_hf, b_f, W_uo, W_ho, b_o, W_hy, b_hy])
                    
                 # LSTM weights 
        update = [(W_ug, W_ug - lr*gW_ug), 
                  (W_hg, W_hg - lr*gW_hg ), 
                  (b_g, b_g - lr*gb_g), 
                  (W_ui, W_ui - lr*gW_ui),
                  (W_hi, W_hi - lr*gW_hi), 
                  (b_i, b_i - lr*gb_i), 
                  (W_uf, W_uf - lr*gW_uf), 
                  (W_hf, W_hf - lr*gW_hf),
                  (b_f, b_f - lr*gb_f),
                  (W_uo, W_uo - lr*gW_uo), 
                  (W_ho, W_ho - lr*gW_ho), 
                  (b_o, b_o - lr*gb_o),
                  (W_hy, W_hy - lr*gW_hy), 
                  (b_hy, b_hy - lr*gb_hy),
                  # FC weights
                  (w_h, w_h - lr * gw_h),
                  (w_h2, w_h2 - lr* gw_h2),
                  (w_h3, w_h3 - lr* gw_h3),
                  (b_h,b_h - lr*gb_h ),
                  (b_h2, b_h2 - lr* gb_h2),
                  (b_h3,b_h3 - lr*gb_h3 )]
        
        #theano.printing.debugprint([h0_tm1], print_type=True)
        self.train_step = theano.function([u, t, lr], cost,
            on_unused_input='warn',
            updates=update,
            allow_input_downcast=True)
        
        
                
        self.predict_step = theano.function([u, t], cost,
           on_unused_input='warn',
           allow_input_downcast=True)
    
    
    def recurrent_fn(self, u_t, h_tm1, s_tm1, W_ug, W_hg, b_g, W_ui, W_hi,
                                            b_i, W_uf, W_hf, b_f, W_uo, W_ho, b_o, W_hy,b_hy):
        
        
        g_t = self.activ2(T.dot(u_t, W_ug) + T.dot(h_tm1, W_hg) + b_g)
        i_t = self.activ1(T.dot(u_t, W_ui) + T.dot(h_tm1, W_hi) + b_i)
        f_t = self.activ1(T.dot(u_t, W_uf) + T.dot(h_tm1, W_hf) + b_f)
        o_t = self.activ1(T.dot(u_t, W_uo) + T.dot(h_tm1, W_ho) + b_o)
        s_t = g_t * i_t + s_tm1*f_t
        h_t = self.activ2(s_t)*o_t
        
        #h_t = self.activ2(T.dot(h_tm1, W_hh) + T.dot(u_t, W_uh) + b_hh)
        return [h_t, s_t]

### Run Model

In [16]:
# LSTM Dimensions
nInputs = 100
nHidden = 20
nOutputs = 1
# FC Network Dimensions
wh_dim = (40, 100)
wh2_dim = (100, 500)
wh3_dim = (500, 100)
lr = 0.001
e = 1.0


rnn = Stacked_Network(nInputs, 
                      nHidden, 
                      nOutputs,
                      wh_dim,
                     wh2_dim,
                     wh3_dim)

### Train Model

In [17]:
train_mse = []
train_error_smooth = []

train_length = len(Xtrain)
test_length = len(Xtest)

# train weights
for j in xrange(train_length):
    u = Xtrain[j].reshape((1,40))
    t = ytrain[j]

    c = rnn.train_step(u, t, lr)
    if j%200==0: print "train iteration {0}: {1}".format(j, np.sqrt(c))
    e = 0.1*np.sqrt(c) + 0.9*e
    # for taining modification, do not smooth the error
    train_mse.append(np.sqrt(c))
    train_error_smooth.append(e)

train iteration 0: 55.7000114639
train iteration 200: 26.7819413457
train iteration 400: 17.9593297174
train iteration 600: 8.91778693751
train iteration 800: 21.4990461026
train iteration 1000: 24.2213266143
train iteration 1200: 44.5800787993
train iteration 1400: 16.5213591366
train iteration 1600: 1.53061495654
train iteration 1800: 15.9178942207
train iteration 2000: 36.2789606436
train iteration 2200: 15.1863230194
train iteration 2400: 12.2493915309
train iteration 2600: 23.9119233509


### Test Model

In [18]:
test_mse = []
test_error_smooth = []

# make predictions                       
for k in xrange(test_length):
    u = Xtest[k].reshape((1,40))
    t = Ytest[k]

    c = rnn.predict_step(u, t)
    if k%200==0: print "test iteration {0}: {1}".format(k, np.sqrt(c))
    e = 0.1*np.sqrt(c) + 0.9*e
    # for taining modification, do not smooth the error
    test_mse.append(np.sqrt(c))
    test_error_smooth.append(e)

test iteration 0: 87.7121614986
test iteration 200: 1.88783826083
test iteration 400: 27.1878382554


### Score Model

In [19]:
get_Strong_Error_Percentage(test_mse)

0.3670886075949367